In [1]:
from google.colab import files
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Bot_sides')

Mounted at /content/drive


In [2]:
!pip install tflearn

     |████████████████████████████████| 112kB 11.4MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-cp36-none-any.whl size=127301 sha256=ab1f0c759dc712a622d25905fe4748be09916bc053d1fe3351bf2bacaed05c35
  Stored in directory: /root/.cache/pip/wheels/31/d2/ed/fb9a0d301dd9586c11e9547120278e624227f22fd5f4baf744
Successfully built tflearn


In [3]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [4]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

In [5]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [6]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [7]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

29 documents
6 classes ['about', 'goodbye', 'greeting', 'help', 'name', 'thanks']
43 unique lemmatized words ["'s", ',', 'anyone', 'appreciate', 'are', 'bye', 'call', 'can', 'do', 'for', 'go', 'gon', 'goodbye', 'got', 'hay', 'hello', 'help', 'helpful', 'hey', 'hi', 'how', 'i', 'is', 'later', 'me', 'na', 'name', 'now', 'purpose', 's', 'see', 'should', 'ta', 'thank', 'thanks', 'that', 'the', 'there', 'what', 'who', 'you', 'your', '’']


In [8]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [9]:
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


Possible Model 1

In [11]:
#equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("model created")

Epoch 1/200
6/6 [==============================] - 2s 4ms/step - loss: 1.8000 - accuracy: 0.2906
Epoch 2/200
6/6 [==============================] - 0s 3ms/step - loss: 1.8016 - accuracy: 0.1293
Epoch 3/200
6/6 [==============================] - 0s 4ms/step - loss: 1.7408 - accuracy: 0.2547
Epoch 4/200
6/6 [==============================] - 0s 4ms/step - loss: 1.6488 - accuracy: 0.2620
Epoch 5/200
6/6 [==============================] - 0s 5ms/step - loss: 1.5709 - accuracy: 0.4692
Epoch 6/200
6/6 [==============================] - 0s 4ms/step - loss: 1.5523 - accuracy: 0.4897
Epoch 7/200
6/6 [==============================] - 0s 4ms/step - loss: 1.4761 - accuracy: 0.5157
Epoch 8/200
6/6 [==============================] - 0s 4ms/step - loss: 1.2732 - accuracy: 0.6216
Epoch 9/200
6/6 [==============================] - 0s 4ms/step - loss: 1.4001 - accuracy: 0.4559
Epoch 10/200
6/6 [==============================] - 0s 4ms/step - loss: 1.1589 - accuracy: 0.8098
Epoch 11/200
6/6 [===========

In [12]:
from keras.layers import Dense, LSTM, Bidirectional, Embedding, Dropout

Possible Model 2

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 128, input_length = max_length,  trainable = False))
model.add(Bidirectional(GRU(128)))
model.add(Dense(64, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation = "relu"))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(21, activation = "softmax"))  
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("model created")

Possible Model 3

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 128, input_length = max_length, trainable = False))
model.add(Bidirectional(LSTM(128)))
model.add(Dense(32, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(21, activation = "softmax"))